In [42]:
import logging
import sys
from pathlib import Path
import itertools

import click
import numpy as np
import pandas as pd
import rasterra as rt
from rasterio.features import rasterize
from rra_tools import jobmon
import xarray as xr
import tqdm
import geopandas as gpd

from spatial_temp_cgf import paths, utils
from spatial_temp_cgf.data_prep.location_mapping import FHS_SHAPE_PATH
from spatial_temp_cgf import cli_options as clio
from spatial_temp_cgf.data import DEFAULT_ROOT, ClimateMalnutritionData

from spatial_temp_cgf.inference.run_inference import get_intercept_raster

In [43]:
output_dir = Path(DEFAULT_ROOT)
measure = 'stunting'
model_version = "2024_07_01.14"
cmip6_scenario = 'ssp126'
year = 2025

In [44]:
cm_data = ClimateMalnutritionData(output_dir / measure)
spec = cm_data.load_model_specification(model_version)
print('loading raster template and shapes')
raster_template = cm_data.load_raster_template()
fhs_shapes = cm_data.load_fhs_shapes()
print('loading models')
models = cm_data.load_model_family(model_version)

loading raster template and shapes
loading models


In [51]:
model = models[0]['model']

coefs = model.coefs['Estimate']
ranefs = model.ranef

predictor = spec.predictors[1]

ldi_spec = None

partial_estimates = {}
if predictor.name == 'intercept':
    r = get_intercept_raster(predictor, coefs, ranefs, fhs_shapes, raster_template)
elif predictor.name == 'ldi_pc_pd':
    ldi_spec = predictor  # deal with later
else:
    transform = predictor.transform
    variable = transform.from_column if hasattr(transform, 'from_column') else predictor.name    
    ds = cm_data.load_climate_raster(variable, cmip6_scenario, year)
    v = utils.xarray_to_raster(ds, nodata=np.nan).resample_to(raster_template)
    


In [54]:
model.var_info

{'intercept': {'transformer': <spatial_temp_cgf.transforms.scaling.Scaler at 0x7f8f8aa30500>},
 'days_over_30C': {'transformer': <spatial_temp_cgf.transforms.scaling.Scaler at 0x7f8f8aa303b0>},
 'ldi_pc_pd': {'transformer': <spatial_temp_cgf.transforms.scaling.Scaler at 0x7f8f58461f10>},
 'elevation': {'transformer': <spatial_temp_cgf.transforms.scaling.Scaler at 0x7f8f8aa30110>},
 'mean_temperature': {'transformer': <spatial_temp_cgf.transforms.scaling.Scaler at 0x7f8f8aa2ff80>},
 'any_days_over_30C': {'transformer': <spatial_temp_cgf.transforms.masking.Masker at 0x7f8f9175fe30>}}